# Buổi 5 – Điều phối đa tác nhân

Trình bày một quy trình đơn giản gồm hai tác nhân (Nhà nghiên cứu -> Biên tập viên) sử dụng Foundry Local.


### Giải thích: Cài đặt phụ thuộc
Cài đặt `foundry-local-sdk` và `openai` cần thiết để truy cập mô hình cục bộ và hoàn thành trò chuyện. Tính chất bất biến.


# Kịch bản
Triển khai mô hình điều phối tối giản với hai tác nhân:
- **Tác nhân Nghiên cứu** thu thập các điểm chính xác ngắn gọn
- **Tác nhân Biên tập** viết lại để rõ ràng và phù hợp với lãnh đạo

Minh họa việc chia sẻ bộ nhớ cho mỗi tác nhân, truyền tuần tự kết quả trung gian, và một hàm pipeline đơn giản. Có thể mở rộng cho nhiều vai trò hơn (ví dụ: Nhà phê bình, Người xác minh) hoặc các nhánh song song.

**Biến môi trường:**
- `FOUNDRY_LOCAL_ALIAS` - Mô hình mặc định sử dụng (mặc định: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Mô hình tác nhân chính (ghi đè ALIAS)
- `AGENT_MODEL_EDITOR` - Mô hình tác nhân biên tập (mặc định là mô hình chính)

**Tham khảo SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Cách hoạt động:**
1. **FoundryLocalManager** tự động khởi động dịch vụ Foundry Local
2. Tải xuống và nạp mô hình được chỉ định (hoặc sử dụng phiên bản đã lưu trong bộ nhớ cache)
3. Cung cấp một điểm cuối tương thích với OpenAI để tương tác
4. Mỗi tác nhân có thể sử dụng một mô hình khác nhau cho các nhiệm vụ chuyên biệt
5. Logic thử lại tích hợp xử lý lỗi tạm thời một cách hiệu quả

**Các tính năng chính:**
- ✅ Tự động phát hiện và khởi tạo dịch vụ
- ✅ Quản lý vòng đời mô hình (tải xuống, lưu cache, nạp)
- ✅ Tương thích với SDK OpenAI cho API quen thuộc
- ✅ Hỗ trợ đa mô hình cho chuyên môn hóa tác nhân
- ✅ Xử lý lỗi mạnh mẽ với logic thử lại
- ✅ Suy luận cục bộ (không cần API đám mây)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Giải thích: Nhập lõi & Gõ kiểu
Giới thiệu dataclasses để lưu trữ tin nhắn của agent và gợi ý kiểu để rõ ràng hơn. Nhập Foundry Local manager + OpenAI client cho các hành động tiếp theo của agent.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Giải thích: Khởi tạo Mô hình (Mẫu SDK)
Sử dụng Foundry Local Python SDK để quản lý mô hình một cách mạnh mẽ:
- **FoundryLocalManager(alias)** - Tự động khởi động dịch vụ và tải mô hình theo alias
- **get_model_info(alias)** - Chuyển đổi alias thành ID mô hình cụ thể
- **manager.endpoint** - Cung cấp endpoint dịch vụ cho OpenAI client
- **manager.api_key** - Cung cấp API key (tùy chọn khi sử dụng cục bộ)
- Hỗ trợ các mô hình riêng biệt cho các agent khác nhau (chính và chỉnh sửa)
- Logic thử lại tích hợp với backoff lũy tiến để tăng độ bền
- Xác minh kết nối để đảm bảo dịch vụ sẵn sàng

**Mẫu SDK chính:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Quản lý vòng đời:**
- Các manager được lưu trữ toàn cục để đảm bảo dọn dẹp đúng cách
- Mỗi agent có thể sử dụng một mô hình khác nhau để chuyên môn hóa
- Tự động phát hiện dịch vụ và xử lý kết nối
- Thử lại một cách nhẹ nhàng với backoff lũy tiến khi gặp lỗi

Điều này đảm bảo khởi tạo đúng cách trước khi bắt đầu điều phối agent.

**Tham khảo:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Giải thích: Lớp Agent & Memory
Định nghĩa `AgentMsg` nhẹ cho các mục trong bộ nhớ và `Agent` bao gồm:
- **Vai trò hệ thống** - Nhân vật và hướng dẫn của Agent
- **Lịch sử tin nhắn** - Duy trì ngữ cảnh cuộc trò chuyện
- **Phương thức act()** - Thực hiện hành động với xử lý lỗi phù hợp

Agent có thể sử dụng các mô hình khác nhau (chính vs chỉnh sửa) và duy trì ngữ cảnh riêng biệt cho mỗi agent. Mô hình này cho phép:
- Duy trì bộ nhớ qua các hành động
- Gán mô hình linh hoạt cho từng agent
- Cách ly lỗi và phục hồi
- Dễ dàng liên kết và điều phối


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Giải thích: Quy trình phối hợp
Tạo ra hai tác nhân chuyên biệt:
- **Nhà nghiên cứu**: Sử dụng mô hình chính, thu thập thông tin thực tế
- **Biên tập viên**: Có thể sử dụng mô hình riêng (nếu được cấu hình), chỉnh sửa và viết lại

Hàm `pipeline`:
1. Nhà nghiên cứu thu thập thông tin thô
2. Biên tập viên chỉnh sửa thành nội dung sẵn sàng cho lãnh đạo
3. Trả về cả kết quả trung gian và kết quả cuối cùng

Mô hình này cho phép:
- Chuyên môn hóa mô hình (các mô hình khác nhau cho các vai trò khác nhau)
- Cải thiện chất lượng thông qua xử lý nhiều giai đoạn
- Khả năng truy xuất nguồn gốc của quá trình chuyển đổi thông tin
- Dễ dàng mở rộng thêm nhiều tác nhân hoặc xử lý song song


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Giải thích: Thực thi & Kết quả của Pipeline
Thực thi pipeline đa tác nhân trên một câu hỏi chủ đề tuân thủ + độ trễ để minh họa:
- Chuyển đổi thông tin qua nhiều giai đoạn
- Chuyên môn hóa và hợp tác giữa các tác nhân
- Cải thiện chất lượng đầu ra thông qua tinh chỉnh
- Khả năng truy xuất nguồn gốc (bảo lưu cả đầu ra trung gian và cuối cùng)

**Cấu trúc Kết quả:**
- `question` - Câu hỏi gốc của người dùng
- `research` - Kết quả nghiên cứu thô (các gạch đầu dòng thực tế)
- `final` - Tóm tắt điều hành đã được tinh chỉnh
- `models` - Các mô hình được sử dụng cho từng giai đoạn

**Ý tưởng Mở rộng:**
1. Thêm một tác nhân Phê bình để đánh giá chất lượng
2. Triển khai các tác nhân nghiên cứu song song cho các khía cạnh khác nhau
3. Thêm một tác nhân Xác minh để kiểm tra tính chính xác
4. Sử dụng các mô hình khác nhau cho các mức độ phức tạp khác nhau
5. Triển khai các vòng phản hồi để cải thiện lặp đi lặp lại


### Nâng cao: Cấu hình Tác nhân Tùy chỉnh

Hãy thử tùy chỉnh hành vi của tác nhân bằng cách thay đổi các biến môi trường trước khi chạy ô khởi tạo:

**Các mô hình có sẵn:**
- Sử dụng `foundry model ls` trong terminal để xem tất cả các mô hình có sẵn
- Ví dụ: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, v.v.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
